In [ ]:
#import csv
import json
import requests
import pandas as pd
from tqdm import tqdm  # progress_apply
import postgis as pg
from pymeos import *

# from tqdm.notebook import tqdm
import asyncio
import aiohttp
# -----------------------
# CONFIGURATION
# -----------------------
SERVER_URL = "http://localhost:8080"
COLLECTION_ID = "ships"           # Change to your collection name


LIMIT_ROWS = None
BATCH_SIZE = 50


# based on PYMEOS example : https://github.com/MobilityDB/PyMEOS-Examples/blob/main/PyMEOS_Examples/AIS.ipynb
# -----------------------
# HELPER FUNCTIONS
# -----------------------

# rework the data using pandas and pymeos to get temporal geometry (trip)

In [ ]:
tqdm.pandas()

pymeos_initialize()
# %%time
# get data, cleanup remove duplicates, wrong lat,long ///



ais = pd.read_csv(
    "./data/aisdk-2024-03-01.zip",
    usecols=["# Timestamp", "MMSI", "Latitude", "Longitude", "SOG"],
    nrows=100
)
ais.columns = ["t", "mmsi", "lat", "lon", "sog"]
# data preparation:  remove rows with no timestamps
ais = ais[ais["t"] != 0]
ais["t"] = pd.to_datetime(ais["t"], format='%d/%m/%Y %H:%M:%S')
ais = ais[ais["mmsi"] != 0]
ais = ais.drop_duplicates(["t", "mmsi"])
ais = ais[(ais["lat"] >= 40.18) & (ais["lat"] <= 84.17)]
ais = ais[(ais["lon"] >= -16.1) & (ais["lon"] <= 32.88)]
ais = ais[(ais["sog"] >= 0) & (ais["sog"] <= 1022)]
ais.dropna()
ais.head()

# Now, we will create the PyMEOS object representing
# the position and the SOG.

In [ ]:
ais["instant"] = ais.progress_apply(
    lambda row: TGeogPointInst(
        point=(row["lon"], row["lat"]), timestamp=row["t"]),
    axis=1,
)
ais["sog"] = ais.progress_apply(
    lambda row: TFloatInst(value=row["sog"], timestamp=row["t"]), axis=1
)
# dropt the standard type columns
ais.drop(["lat", "lon"], axis=1, inplace=True)
ais.head()

# Assembling Trips (MEOS Example)
# Now, we will create the trajectory (TGeogPointSeq) and
# the SOG evolution (TFloatSeq) for every ship (identified by the mmsi)
# using the instants we have created.
# ::::::::::::::::::::::::::::stopped here , time limit

In [ ]:

trajectories = (
    ais.groupby("mmsi")
    .aggregate(
        {
            "instant": lambda x: TGeogPointSeq.from_instants(x, upper_inc=True),
            "sog": lambda x: TFloatSeq.from_instants(x, upper_inc=True),
        }
    )
    .rename({"instant": "trajectory"}, axis=1)
)
trajectories["distance"] = trajectories["trajectory"].apply(
    lambda t: t.length())
trajectories.head()

#### Create collection ships

In [ ]:

collection_metadata = {
    "title": "ships",
    "description": "Collection of ship trajectories",
    "updateFrequency": 1,
    "itemType": "Feature"
}

response = requests.post(
    "http://localhost:8080/collections",
    data=json.dumps(collection_metadata),
    headers={'Content-Type': 'application/json'}
)

if response.status_code == 200:
    print("Collection 'ships' created successfully!")
else:
    print(f"Failed to create collection: {response.status_code} - {response.text}")


# ::::::::::::::::::::::::::::::stopped here , time limit
# Storing in MobilityDB: so they use PYMEOS but here use server.py- susceptible d'etre modifié!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

# https://stackoverflow.com/questions/51699817/python-async-post-requests


In [ ]:
import requests
import json


def clean_up_resources():
 
    try:
        pymeos_finalize() 
        print("MEOS resources cleaned up successfully.")
    except Exception as e:
        print(f"Error during resource cleanup: {str(e)}")

server_url = 'http://localhost:8080/collections/{collectionId}/items'

collection_id = 'ships'
trajectories= trajectories.iloc[:2]

for index, row in trajectories.iterrows():
    try:
    
        post_data = {
            "type":"Feature",
            "id": row.name, 
            "temporalGeometry":json.loads(row["trajectory"].as_mfjson())  
        }
        print("fjkdfkjkjh")
        print(row["trajectory"].as_mfjson() )
        
        json_data = json.dumps(post_data)
# post 
        response = requests.post(
            server_url.format(collectionId=collection_id), 
            data=json_data, 
            headers={'Content-Type': 'application/json'}
        )

     
        if response.status_code == 200:
            print(f"Successfully posted feature {row.name}")
        else:
            print(f"Failed to post feature {row.name}: {response.status_code} - {response.text}")
    
    except requests.exceptions.RequestException as e:

        print(f"Request error for feature {row.name}: {str(e)}")
    
    except Exception as e:

        print(f"Unexpected error for feature {row.name}: {str(e)}")

clean_up_resources()
